In [1]:
import cv2
import numpy as np

from sklearn.metrics import pairwise

C:\Users\LENOVO L 340  7LIN\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
background=None
accumulated_weight=0.5

roi_top=20
roi_bottom=300
roi_right=300
roi_left=600    #this can also be detected and then sent to function 


In [3]:
#returns a bg comon value pixels
def calc_accumulated_avg(frame,accumulated_weight):
    global background
    if background is None:
        background=frame.copy().astype('float')
        return None
    
    cv2.accumulateWeighted(frame,background,accumulated_weight)  #empty bg se common bg mean weight wala create kr dega 


In [4]:
def segment(frame,threshold=25):
    diff=cv2.absdiff(background.astype('uint8'),frame)     #detect hand frpm common bg ceeated 
    ret,thresholded=cv2.threshold(diff,threshold,255,cv2.THRESH_BINARY)
    #grab the external countour now after hand chnage pixels are there 
    image,counours,heirarchy=cv2.findContours(thresholded.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

    if len(counours)==0:
        return None
    else:  #largest external countor will be hande 
        handsegment=max(counours,key=cv2.contourArea)
        return ( thresholded,handsegment)
    


In [5]:
#now we ahve the hand ; we have the bg 
#imlent conve hull now
#now how do we find the circle enclosing the count of vertices ?
# most extareme point find -->find all intersections -->we get the centre 
#then we find  distance of fathest point from the centre and using this we are going to draw circle radius jispe fingers cunt hongi
def count_fingers(thresholded,hand_segment):
    #bg me se thresholf changes wala region
    conv_hull=cv2.convexHull(hand_segment)

    #grab the conv points now 
    top=tuple(conv_hull[conv_hull[:,:,1].argmin()[0]]) 
    bottom=tuple(conv_hull[conv_hull[:,:,1].argmax()[0]]) 
    left=tuple(conv_hull[conv_hull[:,:,0].argmin()[0]]) 
    right=tuple(conv_hull[conv_hull[:,:,0].argmax()[0]]) 
    
    cX=(left[0]+right[0]) //2   #we got the centre from the extremen points of hand 
    cy=(top[1]+bottom[1]) //2

    distace=pairwise.euclidean_distances([cX,cy],Y=[left,right,top,bottom])[0]

    max_distance=distace.max()

    #now drwaing the circle sorrounding the hand by  c of hull

    radius=int(0.9*max_distance)
    circumference=(2*np.pi*radius)


    #now the region of interest -->circle
    circular_roi=np.zeros(thresholded[:2],dtype='unit8')
    cv2.circle(circular_roi,(cX,cy),radius,255,10)

    circular_roi=cv2.bitwise_and(thresholded,thresholded,mask=circular_roi)
    image,counours,heirarchy=cv2.findContours(circular_roi.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)

    count=0

    for cnt in counours:
        (x,y,w,h)=cv2.boundingRect(cnt)
        #it must be not much bottom 

        out_of_wrist=(cy+(cy*0.25))> (y+h)
        limit_points=((circumference*0.25)>cnt.shape[0])

        if out_of_wrist and limit_points:
          cnt+=1  
    return cnt


In [6]:
#now start the image send to all  functions above 
cam=cv2.VideoCapture(0)
num_frames=0
while True:
    ret,frame=cam.read()
    fram_copy=frame.copy()
    roi=frame[roi_top:roi_bottom,roi_right:roi_left]
    gray=cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    gray=cv2.GaussianBlur(gray,(7,7),0)
    if num_frames < 60:
        calc_accumulated_avg(gray,accumulated_weight)

        if num_frames <= 59:
            cv2.putText(fram_copy,'WAIT.GETTING_BG',(200,300),cv2.FONT_HERSHEY_SIMPLEX,1,(0,0,255),2)
            cv2.imshow('Finger Count',fram_copy)
    else:
        hand=segment(gray)
        if hand is not None:
            thresholded,hand_segment=hand #touplt extracct
            #draws contors around real hand 
            cv2.drawContours(fram_copy,[hand_segment+(roi_right,roi_top)],-1,(255,0,0),5)
            
            fingers=count_fingers(thresholded,hand_segment)
            cv2.putText(fram_copy,str(fingers),(70,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,255),2) #lasr wal f size
            cv2.imshow('Thresholded',thresholded) #on same to adjuct the bg 
    cv2.rectangle(fram_copy,(roi_left,roi_top),(roi_right,roi_bottom),(0,0,255),5)
    num_frames+=1
    cv2.imshow('Finger Count',fram_copy)
    k= cv2.waitKey (1) & 0xFF
    if k == 27:
        break

cam.release()
cv2.destroyAllWindows()







            


ValueError: not enough values to unpack (expected 3, got 2)

: 